In [ ]:
from IPython import display
import networkx as nx
import matplotlib.pyplot as plt
import random
import collections
from networkx.algorithms import approximation
from statistics import mean 

%matplotlib inline

In [ ]:
def count_triplets(G):
    
    triplets = set()
    nodes = list(G.nodes())
    
    for i in range(len(nodes)-1):
        nodeA = nodes[i];
        nodeB = nodes[i+1];
        
        adjA = list(G.neighbors(nodeA))
        adjB = list(G.neighbors(nodeB))
        
        if nodeA in adjB:
            common = list(set(adjA) & set(adjB));
            for neighbor in common:
                triplet = [nodeA, nodeB, neighbor]
                triplet = tuple(sorted(triplet))
                triplets.add(triplet)
                
    return len(triplets)

def closeness(g):
    return mean(list(nx.closeness_centrality(g).values()))

def square(g):
    return mean(list(nx.square_clustering(g).values()))

In [ ]:
def prune_graph(pruning_threshold=.5, delta=0.1, method='addition', nodes=200,  edge_prob=1, initial_edge_weight=1, iterations=2000):

    # Create graph
    G = nx.erdos_renyi_graph(nodes, edge_prob)

    # Initialize edge weights
    for i in range(nodes):
        for j in range(i+1, nodes):
            G[i][j]['weight'] = initial_edge_weight

    def increase_weight(weight):
        if (method == 'addition'):
            return weight + delta
        else:
            return weight * (1 + delta)

    def decrease_weight(weight):
        if method == 'addition':
            return weight - delta
        else:
            return weight / (1 + delta)


    node = random.choice(list(G.nodes))


    for i in range(iterations):
        # Pick a random neighbor that has at least 2 edges
        neighbors = list(G.neighbors(node))
        neighbor = random.choice(neighbors)
        while len(neighbors) > 1 and len(list(G.neighbors(neighbor))) == 1:
            neighbors.remove(neighbor)
            neighbor = random.choice(neighbors)
        if (len(list(G.neighbors(neighbor))) == 1):
            print("WARNING: All nodes have at most 1 neighbor")
            break

        # Increase weight by threshold
        G[node][neighbor]['weight'] = increase_weight(G[node][neighbor]['weight'])

        # Decrease wight for all other neighbors
        neighbors = list(G.neighbors(node))
        neighbors.remove(neighbor)
        for other_neighbor in neighbors:
            G[node][other_neighbor]['weight'] = decrease_weight(G[node][other_neighbor]['weight'])
            if (G[node][other_neighbor]['weight'] < pruning_threshold):
                prev_weight = G[node][other_neighbor]['weight']
                G.remove_edge(node, other_neighbor)
            if not nx.is_connected(G):
                G.add_edge(node, other_neighbor)
                G[node][other_neighbor]['weight'] = prev_weight

        node = neighbor

        if (i > 100 and i % 30 == 0):
            plt.clf()
            degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
            degreeCount = collections.Counter(degree_sequence)
            deg, cnt = zip(*degreeCount.items())

            fig, ax = plt.subplots()
            plt.bar(deg, cnt, width=0.80, color="b")

            plt.title("Degree Histogram")
            plt.ylabel("Count")
            plt.xlabel("Degree")
            ax.set_xticks([d + 0.4 for d in deg])
            ax.set_xticklabels(deg)

            # draw graph in inset
            d1 = 1
            plt.axes([d1, d1, d1, d1])
            Gcc = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])

            if (nx.algorithms.planarity.check_planarity(G)[0]):
                pos = nx.planar_layout(G)
            else:
                pos = nx.fruchterman_reingold_layout(G)

            plt.axis("off")
            nx.draw_networkx_nodes(G, pos, node_size=20)
            nx.draw_networkx_edges(G, pos, alpha=0.4)
            display.display(plt.gcf())
            display.clear_output(wait=True)
    return G

In [ ]:
def compute_stats(thr, d, met, it, n):
    
    
    pruned_G = prune_graph(pruning_threshold=thr, delta=d, method=met, iterations=it, nodes=n)

    pruned_triplets = count_triplets(pruned_G)

    edge_probability = len(list(pruned_G.edges()))/((n*(n-1))/2)
    sum = 0
    average_clustering = 0
    large_clique_size = 0
    transitivity = 0
    square_G = 0

    iterations = 10

    for i in range(iterations):
        G = nx.erdos_renyi_graph(n, edge_probability)
        sum += count_triplets(G)
        average_clustering += nx.average_clustering(G)
        large_clique_size += approximation.large_clique_size(G)
        transitivity += nx.transitivity(G)
        square_G = square(G)


    expected_triplets = sum/iterations
    average_clustering /= iterations
    large_clique_size /= iterations
    transitivity /= iterations
    square_G /= iterations

    if (expected_triplets != 0):
        print(f"Triplets ratio: {pruned_triplets / expected_triplets}")
    else:
        print(pruned_triplets, expected_triplets)

    if (average_clustering != 0):
        print(f"Average clustering: {nx.average_clustering(pruned_G) / average_clustering}")
    if (large_clique_size != 0):
        print(f"Largest clique: {approximation.large_clique_size(pruned_G) / large_clique_size}")
    if (transitivity != 0):
        print(f"Transitivity: {nx.transitivity(pruned_G) / transitivity}")
    if (square_G != 0):
        print(f"Square probability: {square(pruned_G) / square_G}")

In [ ]:
compute_stats(0.15, 0.05, 'addition', 2000, 500)